In [2]:
import string
import re
import os

from glob import glob
from pycontractions import Contractions

C:\Users\daniele\AppData\Local\conda\conda\envs\ml\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [24]:
commands = {
    '^x (.+)$': 'examine \g<1>',
    '^m$': 'commands',
    '^l$' : 'look',
    '^nw$' : 'northwest',
    '^sw$' : 'southwest',
    '^ne$' : 'northeast',
    '^se$' : 'southeast',
    '^w$' : 'west',
    '^e$' : 'east',
    '^n$' : 'north',
    '^s$' : 'south',
    '^i$' : 'inventory',
    '^z$' : 'wait',
    '^g$' : 'again'            
}

def parse_transcript(infile, outfile, contractions=None):
    # open the file and start stripping all lines not starting with """Floyd |"""
    with open(infile, 'r', encoding='utf-8') as inf, open(outfile, 'w', encoding='utf-8') as outf:
        content = inf.read()
        
        # skip everything up to the last load attempt (if none, just assume the game's already loaded)
        loads = re.finditer(string=content, pattern=r'say(?:s)? \(to Floyd\), "(?:load|load sleepmask) (?:.+)"', flags=re.IGNORECASE)
        load = None
        for load in loads:
            pass            
        if load:
            content = content[load.end():]
        
        # remove version commands 
        content = re.sub(repl='', string=content, pattern=r'say(?:s)? \(to Floyd\), "version".*?Floyd \| >\n', flags=re.IGNORECASE | re.DOTALL)
        
        # replace external commands with inline ones
        content = re.sub(repl='Floyd | > \g<1>\n', string=content, pattern=r'(?:.+) say(?:s)? \(to Floyd\), "(.+)"', flags=re.IGNORECASE)
        content = content.split('\n')
        
        # expand contractions
        if contractions:
            content = contractions.expand_texts(content, precise=True)
        
        ##############################################################
        ### From here on, we consider everything part of the game. ###
        ##############################################################
        print('Parsing gameplay...')
        
        scene = []
        about = False
        for line in content:
            # Strip away about commands
            if about:
                # Either match the explicit "Q" command, or look for the first empty command prompt
                if re.match(string=line, pattern=r'^Floyd \| >(?: q)?$', flags=re.IGNORECASE):
                    print('Line [{}], about=False'.format(line))
                    about = False
                continue
            else:
                if re.match(string=line, pattern=r'^Floyd \| > (about|help|m|commands)$', flags=re.IGNORECASE):
                    print('Line [{}], about=True'.format(line))
                    about = True
                    continue
                    
            # Just strip blank prompts and "press <key> to continue" lines
            if re.match(string=line, pattern=r'^Floyd \| >$'):
                continue
            if re.match(string=line, pattern=r'press (?:.+) to continue', flags=re.IGNORECASE):
                continue
                    
            # Scene descriptions
            match = re.match(string=line, pattern=r'^Floyd \| ([^>#\n]+)', flags=re.IGNORECASE)
            if match:
                scene.append(match.group(1).strip().lower())
            
            # user's commands are appended to the current scene
            match = re.match(string=line, pattern=r'^Floyd \| > (.+)', flags=re.IGNORECASE)        
            if match:
                command = match.group(1).strip().lower()
                # replace shortcuts
                for pattern,repl in commands.items():
                    command = re.sub(string=command, pattern=pattern, repl=repl)
                newline = '{}\n{}\n'.format(' '.join(scene).replace('  ', ' '), command)
                outf.write(newline)
                
                scene = []
                
        # Append the very last scene
        if len(scene) > 0:
            outf.write(' '.join(scene).replace('  ', ' '))
            
        print('Done.')

In [13]:
# infile = 'raw_weishaupt.txt'
# outfile = 'parsed-weishaupt.txt'

cont = Contractions('data/GoogleNews-vectors-negative300.bin')
cont.load_models()

In [28]:
transcripts = glob('data/transcripts/*.txt')
for transcript in transcripts:
    print('Parsing file: [{}]'.format(transcript))
    parsed = 'data/parsed/parsed-{}'.format('-'.join(transcript.split('-')[1:]))
    parse_transcript(transcript, parsed, cont)
    
#     # move parsed files
#     os.rename(transcript, os.path.join('data/done', os.path.basename(transcript)))

Parsing file: [data/transcripts\raw-12heads.txt]
Parsing gameplay...
Done.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/transcripts\\raw-12heads.txt' -> 'data/done\\raw-12heads.txt'